<span style="color: blue; font-size: 20px; font-weight: bold;">Комментарий ревьюера</span>

<span style="color: blue">Привет, Ярослав! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</span>
<details>
	<summary><u>Инструкция по организационным моментам (кликабельно)</u>⤵</summary>
<span style="color: blue">Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</span>


---


<span style="color: blue">синий текст - просто текст комментария</span>

<span style="color: green">✔️ и зеленый текст - все отлично</span>

<span style="color: orange">⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</span>

<span style="color: red">❌ и красный текст - есть недочеты</span>


</details>    
</br>
<span style="color: blue">Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное.</span>

<span style="color: blue">Ответы на мои комментарии лучше тоже помечать.<br>
Например: </span><br><span style="color: purple; font-weight: bold">Комментарий студента</span>

<span style="color: blue; font-weight: bold;">Давай смотреть, что получилось!</span>

# California Housing Price Prediction

**Цель проекта:** Построение и сравнение двух моделей линейной регрессии для предсказания медианной стоимости жилья (`median_house_value`) на данных California Housing 1990.

**Технологический стек:** PySpark 3.x, MLlib, DataFrame API

**Целевая переменная:** `median_house_value`

**Метрики оценки:** RMSE, MAE, R²

**Модели:**
- **Модель 1:** Все признаки (числовые + OHE-кодированный `ocean_proximity`)
- **Модель 2:** Только числовые признаки

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, mean
from pyspark.sql.types import StringType

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler, Imputer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

import pandas as pd

/home/yka/anaconda3/envs/practicum/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 2.0.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке, разделены на функциональные группы пустой строкой.</font>

## Global Constants

In [2]:
RANDOM_SEED = 42
DATA_PATH = "source/housing.csv"

## Инициализация Spark-сессии

In [3]:
spark = (
    SparkSession
    .builder
    .appName("California Housing - Linear Regression")
    .master("local[3]")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.memory", "2g")
    .config("spark.sql.shuffle.partitions", "8")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .getOrCreate()
)

print(f"Spark version: {spark.version}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/17 11:53:13 WARN Utils: Your hostname, EvanderPC, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/17 11:53:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/17 11:53:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 4.0.0


### Результат

SparkSession успешно инициализирована с локальной конфигурацией (3 потока, 6GB driver memory).

## Загрузка и первичный осмотр данных

In [4]:
try:
    df = spark.read.csv(DATA_PATH, header = True, inferSchema = True)
except:
    df = spark.read.csv("/datasets/housing.csv", header = True, inferSchema = True)

In [5]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [6]:
df.show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично!  Выведены типы данных и первые строчки таблицы.</font>

In [7]:
display(df.describe().toPandas())

26/01/17 11:53:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Верно, что в Пандас попадают уже агрегированные данные.</font>

In [8]:
print(f"Количество строк: {df.count()}")
print(f"Количество колонок: {len(df.columns)}")

Количество строк: 20640
Количество колонок: 10


### Результат

Датасет загружен. Схема соответствует ожидаемой: 8 числовых признаков (double), 1 категориальный (string), 1 целевая переменная (double).

## Анализ пропущенных значений

In [9]:
numeric_cols_check = [f.name for f in df.schema.fields if not isinstance(f.dataType, StringType)]
string_cols_check = [f.name for f in df.schema.fields if isinstance(f.dataType, StringType)]

null_exprs = []
for c in numeric_cols_check:
    null_exprs.append(count(when(col(c).isNull() | isnan(c), c)).alias(c))
for c in string_cols_check:
    null_exprs.append(count(when(col(c).isNull(), c)).alias(c))

null_counts = df.select(null_exprs)
display(null_counts.toPandas())

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,0,0,0,0,207,0,0,0,0,0


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Обнаружены пропуски!</font>

<span style="color: blue; font-weight: bold">Комментарий ревьюера : </span> ⚠️\
<span style="color: darkorange"> Строго говоря, медиану мы можем считать только от обучающих данных.\
В PySpark есть Импутеры, их можно добавлять в Пайплайн.</span>

<span style="color: purple; font-weight: bold">Комментарий студента</span>

**Исправлено:** Ручное заполнение пропусков медианой удалено. Теперь используется `Imputer` из PySpark MLlib, который добавлен в Pipeline и обучается только на тренировочных данных. Это исключает утечку данных (data leakage).

### Результат

Пропуски обнаружены в колонке `total_bedrooms` (207 значений). Заполнение пропусков будет выполнено с помощью `Imputer` внутри Pipeline, чтобы медиана вычислялась только на обучающей выборке.

## Определение списков признаков

In [10]:
categorical_cols = ["ocean_proximity"]

numerical_cols = [
    "longitude",
    "latitude",
    "housing_median_age",
    "total_rooms",
    "total_bedrooms_imputed",
    "population",
    "households",
    "median_income"
]

target = "median_house_value"

print(f"Категориальные признаки: {categorical_cols}")
print(f"Числовые признаки: {numerical_cols}")
print(f"Целевая переменная: {target}")

Категориальные признаки: ['ocean_proximity']
Числовые признаки: ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms_imputed', 'population', 'households', 'median_income']
Целевая переменная: median_house_value


### Результат

Списки признаков определены. Целевая переменная исключена из списка признаков. Обратите внимание: `total_bedrooms_imputed` — это имя колонки после применения `Imputer`.

## Разделение на обучающую и тестовую выборки

In [11]:
train_data, test_data = df.randomSplit([0.8, 0.2], seed = RANDOM_SEED)

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

In [12]:
train_count = train_data.count()
test_count = test_data.count()
total_count = train_count + test_count

print(f"Обучающая выборка: {train_count} ({train_count / total_count * 100:.1f}%)")
print(f"Тестовая выборка: {test_count} ({test_count / total_count * 100:.1f}%)")

Обучающая выборка: 16560 (80.2%)
Тестовая выборка: 4080 (19.8%)


### Результат

Данные разделены на обучающую (80%) и тестовую (20%) выборки с фиксированным `RANDOM_SEED = 42` для воспроизводимости. **Важно:** разделение выполнено ДО любых трансформаций, чтобы избежать утечки данных.

## Просмотр уникальных значений категориального признака

In [13]:
df.select("ocean_proximity").distinct().show()

+---------------+
|ocean_proximity|
+---------------+
|     NEAR OCEAN|
|         ISLAND|
|      <1H OCEAN|
|         INLAND|
|       NEAR BAY|
+---------------+



<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️ <br>
<font color='green'>Верно, что применяешь OHE для Линейной Регрессии.</font>

## Создание Pipeline для Модели 1 (все признаки)

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'>Кодирование и Стандартизацию нужно проводить после разделения на выборки трансформерами обучеными на обучающих данных.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='blue'>Можно использовать [Pipeline](https://habr.com/ru/post/522296/)</font>

<span style="color: purple; font-weight: bold">Комментарий студента</span>

**Исправлено:** Все трансформации (Imputer, StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler) теперь объединены в единый Pipeline. Разделение на train/test выполняется ДО создания Pipeline. Pipeline обучается (`fit`) только на тренировочных данных, что полностью исключает утечку данных (data leakage).

In [14]:
imputer = Imputer(
    inputCols = ["total_bedrooms"],
    outputCols = ["total_bedrooms_imputed"],
    strategy = "median"
)

indexer = StringIndexer(
    inputCol = "ocean_proximity",
    outputCol = "ocean_proximity_idx",
    handleInvalid = "keep"
)

encoder = OneHotEncoder(
    inputCol = "ocean_proximity_idx",
    outputCol = "ocean_proximity_ohe",
    dropLast = True
)

numerical_assembler = VectorAssembler(
    inputCols = numerical_cols,
    outputCol = "numerical_features",
    handleInvalid = "keep"
)

scaler = StandardScaler(
    inputCol = "numerical_features",
    outputCol = "numerical_features_scaled",
    withMean = True,
    withStd = True
)

all_features_assembler = VectorAssembler(
    inputCols = ["numerical_features_scaled", "ocean_proximity_ohe"],
    outputCol = "features_all",
    handleInvalid = "keep"
)

lr_all = LinearRegression(
    featuresCol = "features_all",
    labelCol = "median_house_value",
    predictionCol = "prediction_all",
    regParam = 0.01
)

pipeline_all = Pipeline(stages = [
    imputer,
    indexer,
    encoder,
    numerical_assembler,
    scaler,
    all_features_assembler,
    lr_all
])

print("Pipeline для Модели 1 создан.")
print(f"Этапы: {[type(stage).__name__ for stage in pipeline_all.getStages()]}")

Pipeline для Модели 1 создан.
Этапы: ['Imputer', 'StringIndexer', 'OneHotEncoder', 'VectorAssembler', 'StandardScaler', 'VectorAssembler', 'LinearRegression']


### Результат

Pipeline для Модели 1 создан с этапами:
1. **Imputer** — заполнение пропусков медианой (вычисляется на train)
2. **StringIndexer** — преобразование `ocean_proximity` в числовой индекс
3. **OneHotEncoder** — OHE-кодирование категориального признака
4. **VectorAssembler** — сборка числовых признаков в вектор
5. **StandardScaler** — стандартизация числовых признаков
6. **VectorAssembler** — объединение масштабированных числовых и OHE-признаков
7. **LinearRegression** — модель линейной регрессии с регуляризацией (regParam=0.01)

## Обучение Модели 1

In [15]:
model_all = pipeline_all.fit(train_data)

In [16]:
lr_model_all = model_all.stages[-1]
print(f"Коэффициенты модели 1: {lr_model_all.coefficients}")
print(f"Intercept модели 1: {lr_model_all.intercept}")

Коэффициенты модели 1: [-53605.17300751415,-54869.991032304526,13275.040289251223,-7513.85742519107,28868.39047817371,-49650.745963730114,33427.97475083084,72436.53419731578,12507.472427689148,-26619.96910052803,17479.26153120307,7379.132430573913,162240.91365462114]
Intercept модели 1: 205552.11033569011


In [17]:
predictions_all = model_all.transform(test_data)
predictions_all.select("median_house_value", "prediction_all").show(10)

+------------------+------------------+
|median_house_value|    prediction_all|
+------------------+------------------+
|          103600.0|150535.19525634902|
|          106700.0| 217627.6614955553|
|           73200.0|125170.96373121833|
|           90100.0| 195021.5587143444|
|           67000.0|152292.80485485384|
|           86400.0| 186078.3090950679|
|           70500.0|163940.37848860433|
|           85100.0|180011.97785721224|
|           80500.0| 181582.5950584301|
|           96000.0|170317.42311325346|
+------------------+------------------+
only showing top 10 rows


### Результат

Модель 1 (все признаки: числовые + OHE-кодированный `ocean_proximity`) обучена через Pipeline. Все трансформеры обучены на тренировочных данных. Предсказания получены на тестовой выборке.

## Создание Pipeline для Модели 2 (только числовые признаки)

<span style="color: blue; font-weight: bold">Комментарий ревьюера : </span> ⚠️\
<span style="color: darkorange"> `numerical_only_assembler` - не нужен. Скалер выдаёт готовый вектор.</span>

<span style="color: purple; font-weight: bold">Комментарий студента</span>

**Исправлено:** Лишний `VectorAssembler` удалён. Теперь выход `StandardScaler` (`features_numerical_only`) используется напрямую как `featuresCol` для `LinearRegression`.

In [18]:
imputer_2 = Imputer(
    inputCols = ["total_bedrooms"],
    outputCols = ["total_bedrooms_imputed"],
    strategy = "median"
)

numerical_assembler_2 = VectorAssembler(
    inputCols = numerical_cols,
    outputCol = "numerical_features",
    handleInvalid = "keep"
)

scaler_2 = StandardScaler(
    inputCol = "numerical_features",
    outputCol = "features_numerical_only",
    withMean = True,
    withStd = True
)

lr_num = LinearRegression(
    featuresCol = "features_numerical_only",
    labelCol = "median_house_value",
    predictionCol = "prediction_num",
    regParam = 0.01
)

pipeline_num = Pipeline(stages = [
    imputer_2,
    numerical_assembler_2,
    scaler_2,
    lr_num
])

print("Pipeline для Модели 2 создан.")
print(f"Этапы: {[type(stage).__name__ for stage in pipeline_num.getStages()]}")

Pipeline для Модели 2 создан.
Этапы: ['Imputer', 'VectorAssembler', 'StandardScaler', 'LinearRegression']


### Результат

Pipeline для Модели 2 создан с этапами:
1. **Imputer** — заполнение пропусков медианой
2. **VectorAssembler** — сборка числовых признаков в вектор
3. **StandardScaler** — стандартизация (выход используется напрямую как features)
4. **LinearRegression** — модель линейной регрессии с регуляризацией (regParam=0.01)

## Обучение Модели 2

In [19]:
model_num = pipeline_num.fit(train_data)

In [20]:
lr_model_num = model_num.stages[-1]
print(f"Коэффициенты модели 2: {lr_model_num.coefficients}")
print(f"Intercept модели 2: {lr_model_num.intercept}")

Коэффициенты модели 2: [-85346.34258290856,-91314.18863638745,14233.172317804969,-11463.122268839223,32852.98027905642,-50250.3952904399,33862.920291080096,74230.51348567121]
Intercept модели 2: 205596.3980676288


In [21]:
predictions_num = model_num.transform(test_data)
predictions_num.select("median_house_value", "prediction_num").show(10)

+------------------+------------------+
|median_house_value|    prediction_num|
+------------------+------------------+
|          103600.0|100751.62819965834|
|          106700.0| 190789.9911644132|
|           73200.0|  74735.1749868445|
|           90100.0|162343.73090071618|
|           67000.0|119470.37450376325|
|           86400.0|155923.36849880053|
|           70500.0| 131197.3884082705|
|           85100.0|150449.15803565722|
|           80500.0|150168.96489164347|
|           96000.0|133760.90806567276|
+------------------+------------------+
only showing top 10 rows


### Результат

Модель 2 (только числовые признаки) обучена через Pipeline. Предсказания получены на тестовой выборке.

## Оценка качества моделей

In [22]:
evaluator_rmse_all = RegressionEvaluator(
    labelCol = "median_house_value",
    predictionCol = "prediction_all",
    metricName = "rmse"
)
evaluator_mae_all = RegressionEvaluator(
    labelCol = "median_house_value",
    predictionCol = "prediction_all",
    metricName = "mae"
)
evaluator_r2_all = RegressionEvaluator(
    labelCol = "median_house_value",
    predictionCol = "prediction_all",
    metricName = "r2"
)

rmse_all = evaluator_rmse_all.evaluate(predictions_all)
mae_all = evaluator_mae_all.evaluate(predictions_all)
r2_all = evaluator_r2_all.evaluate(predictions_all)

print("Модель 1 (все признаки):")
print(f"  RMSE: {rmse_all:,.2f}")
print(f"  MAE:  {mae_all:,.2f}")
print(f"  R²:   {r2_all:.4f}")

Модель 1 (все признаки):
  RMSE: 70,786.68
  MAE:  50,863.76
  R²:   0.6378


In [23]:
evaluator_rmse_num = RegressionEvaluator(
    labelCol = "median_house_value",
    predictionCol = "prediction_num",
    metricName = "rmse"
)
evaluator_mae_num = RegressionEvaluator(
    labelCol = "median_house_value",
    predictionCol = "prediction_num",
    metricName = "mae"
)
evaluator_r2_num = RegressionEvaluator(
    labelCol = "median_house_value",
    predictionCol = "prediction_num",
    metricName = "r2"
)

rmse_num = evaluator_rmse_num.evaluate(predictions_num)
mae_num = evaluator_mae_num.evaluate(predictions_num)
r2_num = evaluator_r2_num.evaluate(predictions_num)

print("Модель 2 (только числовые):")
print(f"  RMSE: {rmse_num:,.2f}")
print(f"  MAE:  {mae_num:,.2f}")
print(f"  R²:   {r2_num:.4f}")

Модель 2 (только числовые):
  RMSE: 71,791.59
  MAE:  51,804.75
  R²:   0.6275


In [24]:
results_df = pd.DataFrame({
    "Модель": ["Модель 1 (все признаки)", "Модель 2 (числовые)"],
    "RMSE": [rmse_all, rmse_num],
    "MAE": [mae_all, mae_num],
    "R²": [r2_all, r2_num]
})

display(results_df)

,Модель,RMSE,MAE,R²
0,Модель 1 (все признаки),70786.679706,50863.756294,0.637843
1,Модель 2 (числовые),71791.593718,51804.750819,0.627487


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично, что есть табличка результатов для сравнения.</font>

### Результат

Метрики RMSE, MAE и R² рассчитаны для обеих моделей и представлены в сравнительной таблице.

## Сравнительный анализ

In [25]:
rmse_diff = rmse_num - rmse_all
rmse_improvement = (rmse_diff / rmse_num) * 100

r2_diff = r2_all - r2_num

print("Сравнительный анализ:")
print(f"  Разница RMSE: {rmse_diff:,.2f} (улучшение {rmse_improvement:.2f}%)")
print(f"  Разница R²: {r2_diff:.4f}")

Сравнительный анализ:
  Разница RMSE: 1,004.91 (улучшение 1.40%)
  Разница R²: 0.0104


## Общий вывод

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'> Очень не хватает общего вывода. В выводах можно приводить полученные ранее значения. Также можно расписать все, что было сделано в ходе проведения работы.</font>

<span style="color: purple; font-weight: bold">Комментарий студента</span>

**Исправлено:** Добавлен развёрнутый общий вывод по проекту.

### Проделанная работа

В рамках данного проекта была решена задача предсказания медианной стоимости жилья в Калифорнии (`median_house_value`) на основе датасета California Housing 1990 с использованием PySpark MLlib.

**Основные этапы:**
1. **Загрузка и исследование данных:** Проанализирован датасет из 20,640 записей с 9 признаками (8 числовых + 1 категориальный `ocean_proximity`).

2. **Обработка пропущенных значений:** Обнаружено 207 пропусков в колонке `total_bedrooms`. Для заполнения использован `Imputer` внутри Pipeline с медианной стратегией, что гарантирует вычисление медианы только на обучающих данных.

3. **Разделение данных:** Датасет разделён на обучающую (80%) и тестовую (20%) выборки ДО применения любых трансформаций для исключения утечки данных.

4. **Построение Pipeline:** Для каждой модели создан отдельный Pipeline, включающий все этапы предобработки (Imputer, StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler) и обучения (LinearRegression с L2-регуляризацией regParam=0.01).

5. **Обучение и оценка моделей:**
   - **Модель 1** (все признаки): использует числовые и OHE-кодированный категориальный признак
   - **Модель 2** (только числовые): использует только числовые признаки

### Результаты

| Метрика | Модель 1 (все признаки) | Модель 2 (числовые) |
|---------|------------------------|--------------------|
| RMSE    | ~70,000-71,000         | ~71,000-72,000     |
| MAE     | ~50,000-51,000         | ~51,000-52,000     |
| R²      | ~0.63-0.64             | ~0.62-0.63         |

**Влияние категориального признака:**
Добавление OHE-кодированного признака `ocean_proximity` (близость к океану) улучшает качество модели примерно на 1-1.5% по RMSE и ~1% по R². Это логично, поскольку расположение относительно океана является значимым фактором стоимости недвижимости в Калифорнии.

### Рекомендации по улучшению

1. **Feature Engineering:**
   - Создание производных признаков: `rooms_per_household = total_rooms / households`, `bedrooms_ratio = total_bedrooms / total_rooms`
   - Бинаризация или полиномиальное преобразование признаков

2. **Подбор гиперпараметров:**
   - Использование CrossValidator для оптимизации `regParam` и `elasticNetParam`
   - Эксперименты с различными типами регуляризации (L1, L2, ElasticNet)

3. **Альтернативные модели:**
   - Gradient Boosted Trees (GBTRegressor)
   - Random Forest (RandomForestRegressor)
   - Эти модели лучше улавливают нелинейные зависимости

### Ограничения линейной регрессии

R² около 0.63 указывает на то, что линейная модель объясняет лишь ~63% дисперсии целевой переменной. Это может быть связано с:
- Нелинейными зависимостями между признаками и целевой переменной
- Пространственной автокорреляцией (соседние районы имеют схожие цены)
- Ограничением целевой переменной сверху (значения capped на 500,001)

In [26]:
spark.stop()

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Здорово, что не забываешь останавливать сессию!</font>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'> Ярослав, хороший проект получился!
Большое спасибо за проделанную работу. Видно, что приложено много усилий.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Стандартизация и Кодировка для train и test по отдельности, обработчиком обученным на train.</font></ul>
<ul><font color='red'>Добавь Общий вывод.</font></ul>

<font color='blue'>Что можно сделать лучше:</font>
<font color='orange'>В работе я оставил несколько советов. Буду рад, если ты учтешь их.</font></ul>

<font color='blue'><b>Жду новую версию проекта :)</b></font>